In [38]:
import nse_ind
import volatility_underlying
import numpy as np
from math import log,exp,sqrt,pi
from scipy.stats import norm
from datetime import datetime
import pandas as pd

nse=nse_ind.NSE()

# print(nse.pre_market_data("NIFTY 50"))
df=nse.pre_market_data("Securities in F&O")

In [39]:
# risk-free rate of return
# choosing indian bond 1Y return and approximating it to daily return
r=0.0685/365

In [40]:
# Extracts the stock names which have derivatives
stocks_with_derivatives=[]
for i in df.index:
    stocks_with_derivatives.append(i)

stocks_symbols=[stock+'.NS' for stock in stocks_with_derivatives]

In [41]:
# Underlying price (per share): S; 
# Strike price of the option (per share): K;
# Time to maturity (years): T;
# Continuously compounding risk-free interest rate: r;
# Volatility: sigma;

## define two functions, d1 and d2 in Black-Scholes model
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2)*T)/sigma*sqrt(T)
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

In [42]:
## define the call options price function
def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))

In [43]:
## define the put options price function
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)

In [44]:
undervalued=[]
overvalued=[]
for i in stocks_with_derivatives:
    df=pd.DataFrame(nse.option_data(i))
    vol=volatility_underlying.stock_volatility(i+'.NS')
    volatility=vol.calc_volatility()
    for index,rows in df.iterrows():
        date1=datetime.strptime(rows['expiryDate'],'%d-%b-%Y').date()
        date2=datetime.now().date()
        time_to_maturity=(date1-date2).days
        if time_to_maturity==0:
            time_to_maturity=1
        d1_calc=d1(rows['underlyingValue'],rows['strikePrice'],time_to_maturity/365,r*time_to_maturity,volatility)
        d2_calc=d2(rows['underlyingValue'],rows['strikePrice'],time_to_maturity/365,r*time_to_maturity,volatility)
        if rows['instrumentType']=='PE':
            calc_price=bs_put(rows['underlyingValue'],rows['strikePrice'],time_to_maturity/365,r*time_to_maturity,volatility)
            if calc_price>1.5*rows['askPrice']:
                overvalued.append(index)
            elif calc_price<rows['askPrice']/2:
                undervalued.append(index)
        else:
            calc_price=bs_call(rows['underlyingValue'],rows['strikePrice'],time_to_maturity/365,r*time_to_maturity,volatility)
            if calc_price>1.5*rows['askPrice']:
                overvalued.append(index)
            elif calc_price<rows['askPrice']/2:
                undervalued.append(index)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [48]:
undervalued

['OPTSTKIDEA29-08-2024PE8.00',
 'OPTSTKIDEA31-10-2024PE10.00',
 'OPTSTKIDEA31-10-2024CE10.00',
 'OPTSTKIDEA29-08-2024PE12.00',
 'OPTSTKIDEA26-09-2024PE12.00',
 'OPTSTKIDEA31-10-2024PE12.00',
 'OPTSTKIDEA31-10-2024CE12.00',
 'OPTSTKIDEA29-08-2024PE13.00',
 'OPTSTKIDEA26-09-2024PE13.00',
 'OPTSTKIDEA29-08-2024PE14.00',
 'OPTSTKIDEA26-09-2024PE14.00',
 'OPTSTKIDEA31-10-2024CE14.00',
 'OPTSTKIDEA29-08-2024PE15.00',
 'OPTSTKIDEA29-08-2024PE16.00',
 'OPTSTKIDEA29-08-2024CE17.00',
 'OPTSTKIDEA29-08-2024CE18.00',
 'OPTSTKIDEA29-08-2024CE19.00',
 'OPTSTKIDEA26-09-2024CE19.00',
 'OPTSTKIDEA29-08-2024CE20.00',
 'OPTSTKIDEA26-09-2024CE20.00',
 'OPTSTKIDEA29-08-2024CE21.00',
 'OPTSTKIDEA29-08-2024PE21.00',
 'OPTSTKIDEA29-08-2024CE22.00',
 'OPTSTKIDEA26-09-2024CE22.00',
 'OPTSTKIDEA31-10-2024CE22.00',
 'OPTSTKIDEA29-08-2024CE24.00',
 'OPTSTKIDEA26-09-2024CE24.00',
 'OPTSTKBIOCON29-08-2024PE280.00',
 'OPTSTKBIOCON29-08-2024CE280.00',
 'OPTSTKBIOCON26-09-2024PE280.00',
 'OPTSTKBIOCON29-08-2024CE285.00

In [47]:
overvalued

['OPTSTKBIOCON29-08-2024CE315.00',
 'OPTSTKBIOCON29-08-2024CE320.00',
 'OPTSTKBIOCON29-08-2024CE330.00',
 'OPTSTKBIOCON31-10-2024PE330.00',
 'OPTSTKBIOCON29-08-2024CE340.00',
 'OPTSTKBIOCON29-08-2024CE350.00',
 'OPTSTKBIOCON29-08-2024PE350.00',
 'OPTSTKBIOCON31-10-2024CE350.00',
 'OPTSTKBIOCON29-08-2024CE355.00',
 'OPTSTKBIOCON29-08-2024PE355.00',
 'OPTSTKBIOCON31-10-2024CE360.00',
 'OPTSTKBIOCON31-10-2024PE360.00',
 'OPTSTKBIOCON31-10-2024CE370.00',
 'OPTSTKBIOCON26-09-2024PE375.00',
 'OPTSTKBIOCON31-10-2024CE380.00',
 'OPTSTKBIOCON31-10-2024CE400.00',
 'OPTSTKRECLTD31-10-2024PE520.00',
 'OPTSTKRECLTD26-09-2024PE580.00',
 'OPTSTKRECLTD26-09-2024PE590.00',
 'OPTSTKRECLTD26-09-2024PE600.00',
 'OPTSTKRECLTD31-10-2024CE600.00',
 'OPTSTKRECLTD31-10-2024PE600.00',
 'OPTSTKRECLTD26-09-2024PE610.00',
 'OPTSTKRECLTD29-08-2024PE620.00',
 'OPTSTKRECLTD26-09-2024PE620.00',
 'OPTSTKRECLTD31-10-2024CE620.00',
 'OPTSTKRECLTD29-08-2024CE630.00',
 'OPTSTKRECLTD26-09-2024PE630.00',
 'OPTSTKRECLTD29-08-